# Package for WRC Results Scrape

In [1]:
url_base='http://www.wrc.com/service/sasCacheApi.php?route={stub}'

In [2]:
stubs = { 'itinerary': 'rallies/{rallyId}/itinerary',
          'startlists': 'rallies/{rallyId}/entries',
         'penalties': 'rallies/{rallyId}/penalties',
         'retirements': 'rallies/{rallyId}/retirements',
         'stagewinners':'rallies/{rallyId}/stagewinners',
         'overall':'stages/{stageId}/results?rallyId={rallyId}',
         'split_times':'stages/{stageId}/splittimes?rallyId={rallyId}',
         'stage_times_stage':'stages/{stageId}/stagetimes?rallyId={rallyId}',
         'stage_times_overall':'stages/{stageId}/results?rallyId={rallyId}',
         'championship':'championships/{championshipId}',
         'championship_results':'championships/{championshipId}/results' }

In [3]:
import requests
import re
import json
from bs4 import BeautifulSoup 

import sqlite3

import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
#SQL in wrcResults.sql
q='''
CREATE TABLE "itinerary_event" (
  "eventId" INTEGER,
  "itineraryId" INTEGER PRIMARY KEY,
  "name" TEXT,
  "priority" INTEGER
);
CREATE TABLE "itinerary_legs" (
  "itineraryId" INTEGER,
  "itineraryLegId" INTEGER PRIMARY KEY,
  "legDate" TEXT,
  "name" TEXT,
  "order" INTEGER,
  "startListId" INTEGER,
  "status" TEXT,
  FOREIGN KEY ("itineraryId") REFERENCES "itinerary_event" ("itineraryId")
);
CREATE TABLE "itinerary_sections" (
  "itineraryLegId" INTEGER,
  "itinerarySectionId" INTEGER PRIMARY KEY,
  "name" TEXT,
  "order" INTEGER,
  FOREIGN KEY ("itineraryLegId") REFERENCES "itinerary_legs" ("itineraryLegId")
);
CREATE TABLE "itinerary_stages" (
  "code" TEXT,
  "distance" REAL,
  "eventId" INTEGER,
  "name" TEXT,
  "number" INTEGER,
  "stageId" INTEGER PRIMARY KEY,
  "stageType" TEXT,
  "status" TEXT,
  "timingPrecision" TEXT,
  "itineraryLegId" INTEGER,
  FOREIGN KEY ("itineraryLegId") REFERENCES "itinerary_legs" ("itineraryLegId")
);
CREATE TABLE "itinerary_controls" (
  "code" TEXT,
  "controlId" INTEGER PRIMARY KEY,
  "distance" REAL,
  "eventId" INTEGER,
  "firstCarDueDateTime" TEXT,
  "firstCarDueDateTimeLocal" TEXT,
  "location" TEXT,
  "stageId" INTEGER,
  "status" TEXT,
  "targetDuration" TEXT,
  "targetDurationMs" INTEGER,
  "timingPrecision" TEXT,
  "type" TEXT,
  "itineraryLegId" INTEGER,
  FOREIGN KEY ("itineraryLegId") REFERENCES "itinerary_legs" ("itineraryLegId")
);
CREATE TABLE "startlists" (
  "codriver.abbvName" TEXT,
  "codriver.code" TEXT,
  "codriver.country.countryId" INTEGER,
  "codriver.country.iso2" TEXT,
  "codriver.country.iso3" TEXT,
  "codriver.country.name" TEXT,
  "codriver.countryId" INTEGER,
  "codriver.firstName" TEXT,
  "codriver.fullName" TEXT,
  "codriver.lastName" TEXT,
  "codriver.personId" INTEGER,
  "codriverId" INTEGER,
  "driver.abbvName" TEXT,
  "driver.code" TEXT,
  "driver.country.countryId" INTEGER,
  "driver.country.iso2" TEXT,
  "driver.country.iso3" TEXT,
  "driver.country.name" TEXT,
  "driver.countryId" INTEGER,
  "driver.firstName" TEXT,
  "driver.fullName" TEXT,
  "driver.lastName" TEXT,
  "driver.personId" INTEGER,
  "driverId" INTEGER,
  "eligibility" TEXT,
  "entrant.entrantId" INTEGER,
  "entrant.logoFilename" TEXT,
  "entrant.name" TEXT,
  "entrantId" INTEGER,
  "entryId" INTEGER PRIMARY KEY,
  "eventId" INTEGER,
  "group.name" TEXT,
  "groupId" INTEGER,
  "identifier" TEXT,
  "manufacturer.logoFilename" TEXT,
  "manufacturer.manufacturerId" INTEGER,
  "manufacturer.name" TEXT,
  "manufacturerId" INTEGER,
  "priority" TEXT,
  "status" TEXT,
  "tyreManufacturer" TEXT,
  "vehicleModel" TEXT,
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "startlist_classes" (
  "eventClassId" INTEGER,
  "eventId" INTEGER,
  "name" TEXT,
  "entryId" INTEGER,
  PRIMARY KEY ("eventClassId","entryId"),
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "penalties" (
  "controlId" INTEGER,
  "entryId" INTEGER,
  "penaltyDuration" TEXT,
  "penaltyDurationMs" INTEGER,
  "penaltyId" INTEGER PRIMARY KEY,
  "reason" TEXT,
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "retirements" (
  "controlId" INTEGER,
  "entryId" INTEGER,
  "reason" TEXT,
  "retirementDateTime" TEXT,
  "retirementDateTimeLocal" TEXT,
  "retirementId" INTEGER PRIMARY KEY,
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "stagewinners" (
  "elapsedDuration" TEXT,
  "elapsedDurationMs" INTEGER,
  "entryId" INTEGER,
  "stageId" INTEGER,
  "stageName" TEXT,
  PRIMARY KEY ("stageId","entryId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId"),
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId")
);
CREATE TABLE "stage_overall" (
  "diffFirst" TEXT,
  "diffFirstMs" INTEGER,
  "diffPrev" TEXT,
  "diffPrevMs" INTEGER,
  "entryId" INTEGER,
  "penaltyTime" TEXT,
  "penaltyTimeMs" INTEGER,
  "position" INTEGER,
  "stageTime" TEXT,
  "stageTimeMs" INTEGER,
  "totalTime" TEXT,
  "totalTimeMs" INTEGER,
  "stageId" INTEGER,
  PRIMARY KEY ("stageId","entryId"),
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "split_times" (
  "elapsedDuration" TEXT,
  "elapsedDurationMs" INTEGER,
  "entryId" INTEGER,
  "splitDateTime" TEXT,
  "splitDateTimeLocal" TEXT,
  "splitPointId" INTEGER,
  "splitPointTimeId" INTEGER PRIMARY KEY,
  "startDateTime" TEXT,
  "startDateTimeLocal" TEXT,
  "stageId" INTEGER,
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "stage_times_stage" (
  "diffFirst" TEXT,
  "diffFirstMs" REAL,
  "diffPrev" TEXT,
  "diffPrevMs" REAL,
  "elapsedDuration" TEXT,
  "elapsedDurationMs" REAL,
  "entryId" INTEGER,
  "position" INTEGER,
  "source" TEXT,
  "stageId" INTEGER,
  "stageTimeId" INTEGER PRIMARY KEY,
  "status" TEXT,
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "stage_times_overall" (
  "diffFirst" TEXT,
  "diffFirstMs" INTEGER,
  "diffPrev" TEXT,
  "diffPrevMs" INTEGER,
  "entryId" INTEGER,
  "penaltyTime" TEXT,
  "penaltyTimeMs" INTEGER,
  "position" INTEGER,
  "stageTime" TEXT,
  "stageTimeMs" INTEGER,
  "totalTime" TEXT,
  "totalTimeMs" INTEGER,
  "stageId" INTEGER,
  PRIMARY KEY ("stageId","entryId"),
  FOREIGN KEY ("stageId") REFERENCES "itinerary_stages" ("stageId"),
  FOREIGN KEY ("entryId") REFERENCES "startlists" ("entryId")
);
CREATE TABLE "championship_lookup" (
  "championshipId" INTEGER PRIMARY KEY,
  "fieldFiveDescription" TEXT,
  "fieldFourDescription" TEXT,
  "fieldOneDescription" TEXT,
  "fieldThreeDescription" TEXT,
  "fieldTwoDescription" TEXT,
  "name" TEXT,
  "seasonId" INTEGER,
  "type" TEXT,
  "_codeClass" TEXT,
  "_codeTyp" TEXT
);
CREATE TABLE "championship_results" (
  "championshipEntryId" INTEGER,
  "championshipId" INTEGER,
  "dropped" INTEGER,
  "eventId" INTEGER,
  "pointsBreakdown" TEXT,
  "position" TEXT,
  "publishedStatus" TEXT,
  "status" TEXT,
  "totalPoints" INTEGER,
  PRIMARY KEY ("championshipEntryId","eventId"),
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId"),
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "championship_codrivers" (
  "championshipEntryId" INTEGER PRIMARY KEY,
  "championshipId" INTEGER,
  "entrantId" TEXT,
  "ManufacturerTyre" TEXT,
  "Manufacturer" TEXT,
  "FirstName" TEXT,
  "CountryISO3" TEXT,
  "LastName" TEXT,
  "manufacturerId" INTEGER,
  "personId" INTEGER,
  "tyreManufacturer" TEXT,
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId")
);
CREATE TABLE "championship_manufacturers" (
  "championshipEntryId" INTEGER PRIMARY KEY ,
  "championshipId" INTEGER,
  "entrantId" INTEGER,
  "Name" TEXT,
  "LogoFileName" TEXT,
  "Manufacturer" TEXT,
  "manufacturerId" INTEGER,
  "personId" TEXT,
  "tyreManufacturer" TEXT,
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId")
);
CREATE TABLE "championship_rounds" (
  "championshipId" INTEGER,
  "eventId" INTEGER,
  "order" INTEGER,
  PRIMARY KEY ("championshipId","eventId"),
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId"),
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "championship_events" (
  "categories" TEXT,
  "country.countryId" INTEGER,
  "country.iso2" TEXT,
  "country.iso3" TEXT,
  "country.name" TEXT,
  "countryId" INTEGER,
  "eventId" INTEGER PRIMARY KEY,
  "finishDate" TEXT,
  "location" TEXT,
  "mode" TEXT,
  "name" TEXT,
  "organiserUrl" TEXT,
  "slug" TEXT,
  "startDate" TEXT,
  "surfaces" TEXT,
  "timeZoneId" TEXT,
  "timeZoneName" TEXT,
  "timeZoneOffset" INTEGER,
  "trackingEventId" INTEGER ,
  FOREIGN KEY ("eventId") REFERENCES "itinerary_event" ("eventId")
);
CREATE TABLE "championship_drivers" (
  "championshipEntryId" INTEGER PRIMARY KEY ,
  "championshipId" INTEGER,
  "entrantId" TEXT,
  "ManufacturerTyre" TEXT,
  "Manufacturer" TEXT,
  "FirstName" TEXT,
  "CountryISO3" TEXT,
  "LastName" TEXT,
  "manufacturerId" INTEGER,
  "personId" INTEGER,
  "tyreManufacturer" TEXT,
  FOREIGN KEY ("championshipId") REFERENCES "championship_lookup" ("championshipId")
);
'''
#conn = sqlite3.connect('wrc18_test1keys.db')
#c = conn.cursor()
#c.executescript(q)

In [4]:
meta={'rallyId':None, 'stages':[], 'championshipId':None }

In [5]:
def getRally_URLs(results_main_url=None):
    if results_main_url is None:
        results_main_url='http://www.wrc.com/en/wrc/results/wales/stage-times/page/416-238---.html#'

    html=requests.get(results_main_url)
    soup=BeautifulSoup(html.content, "html5lib")
    #BeautifulSoup has a routine - find_all() - that will find all the HTML tags of a particular sort
    #Links are represented in HTML pages in the form <a href="http//example.com/page.html">link text</a>
    #Grab all the <a> (anchor) tags...
    souplist=soup.findAll("li",{'class':'flag'})

    items={}
    for s in souplist:
        href=s.find('a')['href']
        if href:
            title=s.find('img')['title']
            title = 'Monaco' if title == 'Monte Carlo' else title
            items[title]=href
    return items

def listRallies(display=True, **kwargs):
    rallyURLs = getRally_URLs(**kwargs)
    if display:
        print( ', '.join(rallyURLs.keys()) )
    else:
        return getRallyIDs

In [7]:
#listRallies()

Monaco, Sweden, Mexico, France, Argentina, Portugal, Italy, Finland, Germany, Turkey, Wales, Spain, Australia


In [8]:
def _getRallyID(rallyURL):
    html=requests.get(rallyURL)
    m = re.search("var rallyId = '(.+?)'", html.text)
    if m:
        return m.group(1)
    return None
            
def getRallyIDs(rally=None,results_main_url=None):
    rallyids={}

    items = getRally_URLs(results_main_url)
    
    #if we know the rally, just get that one.. 
    if rally in items:
        items = {rally:items[rally]}

    for item in items:
        rallyids[item] = _getRallyID(items[item])

    return rallyids 

In [25]:
#rallyIDs = getRallyIDs()

In [9]:
def set_rallyId(meta, rally, rallyIDs=None):
    if rallyIDs is None:
        rallyIDs = getRallyIDs()
    if rally in rallyIDs:
        meta['rallyId']=rallyIDs[rally]
        meta['rally_name'] = rally

In [10]:
def getItinerary(meta):
    itinerary_json=requests.get( url_base.format(stub=stubs['itinerary'].format(**meta) ) ).json()
    itinerary_event = json_normalize(itinerary_json).drop('itineraryLegs', axis=1)
    
    #meta='eventId' for eventId
    itinerary_legs = json_normalize(itinerary_json, 
                                    record_path='itineraryLegs').drop('itinerarySections', axis=1)
    #meta='eventId' for eventId
    itinerary_sections = json_normalize(itinerary_json,
                                        ['itineraryLegs', 'itinerarySections']).drop(['stages','controls'],axis=1)

    itinerary_stages=json_normalize(itinerary_json['itineraryLegs'],
                                    ['itinerarySections','stages'],
                                    meta = 'itineraryLegId')
    meta['stages']=itinerary_stages['stageId'].tolist()
    itinerary_controls=json_normalize(itinerary_json['itineraryLegs'], 
                                  ['itinerarySections','controls'] , meta='itineraryLegId')
    itinerary_controls['stageId'] = itinerary_controls['stageId'].fillna(-1).astype(int)
    
    return itinerary_event, itinerary_legs, itinerary_sections, itinerary_stages, itinerary_controls

In [11]:
#a,b,c,d,e = getItinerary(meta)

In [12]:
def _get_single_json_table(meta, stub):
    _json = requests.get( url_base.format(stub=stubs[stub].format(**meta) ) ).json()
    return json_normalize(_json)

In [13]:
def get_startlists(meta):
    startlists_json=requests.get( url_base.format(stub=stubs['startlists'].format(**meta) ) ).json()
    startlists = json_normalize(startlists_json).drop('eventClasses', axis=1)
    startlist_classes = json_normalize(startlists_json,['eventClasses'], 'entryId' )
    
    return startlists, startlist_classes 

In [14]:
def get_penalties(meta):
    penalties = _get_single_json_table(meta, 'penalties')
    return penalties

In [15]:
def get_retirements(meta):
    retirements = _get_single_json_table(meta, 'retirements')
    return retirements

In [16]:
def get_stagewinners(meta):
    stagewinners = _get_single_json_table(meta, 'stagewinners')
    return stagewinners

In [17]:
def _stage_iterator(meta, stub):
    meta2={'rallyId':meta['rallyId']}
    df = pd.DataFrame()
    for stageId in meta['stages']:
        meta2['stageId']=stageId
        _json=requests.get( url_base.format(stub=stubs[stub].format(**meta2) ) ).json()
        _df = json_normalize(_json)
        _df['stageId'] = stageId
        df = pd.concat([df, _df])
    return df

In [18]:
def get_overall(meta):
    stage_overall = _stage_iterator(meta, 'overall')
    return stage_overall

In [19]:
def get_splitTimes(meta):
    split_times = _stage_iterator(meta, 'split_times')
    return split_times

In [20]:
def get_stage_times_stage(meta):
    stage_times_stage = _stage_iterator(meta, 'stage_times_stage')
    return stage_times_stage

In [21]:
def get_stage_times_overall(meta):
    stage_times_overall = _stage_iterator(meta, 'stage_times_overall')
    return stage_times_overall

In [22]:
def _get_championship_codes(url=None):
    if url is None:
        url = 'http://www.wrc.com/en/wrc/results/championship-standings/page/4176----.html'
    html2=requests.get(url).text
    m = re.search("var championshipClasses = (.*?);", html2, re.DOTALL)
    mm=m.group(1).replace('\n','').replace("'",'"')
    d=json.loads(mm)
    #https://stackoverflow.com/a/35758583/454773
    championshipClasses={k.replace(' ', ''): v for k, v in d.items()}
    return championshipClasses

In [73]:
#_get_championship_codes()

{'JWRC': {'Co-Driver': '16', 'Driver': '15'},
 'WRC': {'Co-Driver': '7', 'Driver': '6', 'Manufacturers': '8'},
 'WRC2': {'Co-Driver': '11', 'Driver': '10', 'Manufacturers': '9'},
 'WRC3': {'Co-Driver': '14', 'Driver': '13', 'Manufacturers': '12'}}

In [30]:
def championship_tables(champ_class=None, champ_typ=None):
    #if championship is None then get all
    championship_lookup = pd.DataFrame()
    championship_entries_all = {}
    championship_rounds = pd.DataFrame()
    championship_events = pd.DataFrame()
    championship_results = pd.DataFrame()
    
    championship_codes = _get_championship_codes()
    _class_codes = championship_codes.keys() if champ_class is None else [champ_class]
    for champClass in _class_codes:
        _champ_typ = championship_codes[champClass].keys() if champ_typ is None else [champ_typ]
        for champType in _champ_typ:
            if champType not in championship_entries_all:
                championship_entries_all[champType] = pd.DataFrame()
            
            champ_num = championship_codes[champClass][champType]
            meta2={'championshipId': champ_num}
            
            championship_json=requests.get( url_base.format(stub=stubs['championship'].format(**meta2) ) ).json()

            _championship_lookup = json_normalize(championship_json).drop(['championshipEntries','championshipRounds'], axis=1)
            _championship_lookup['_codeClass'] = champClass
            _championship_lookup['_codeTyp'] = champType
            championship_lookup = pd.concat([championship_lookup,_championship_lookup])
            
            championships={}
            championship_dict = _championship_lookup.to_dict()
            championships[champ_num] = {c:championship_dict[c][0] for c in championship_dict}
            renamer={c.replace('Description',''):championships[champ_num][c] for c in championships[champ_num] if c.startswith('field')}            
            _championship_entries = json_normalize(championship_json,['championshipEntries'] )
            _championship_entries = _championship_entries.rename(columns=renamer)
            _championship_entries = _championship_entries[[c for c in _championship_entries.columns if c!='']]
            championship_entries_all[champType] = pd.concat([championship_entries_all[champType],_championship_entries])
            
            _championship_rounds = json_normalize(championship_json,['championshipRounds'] ).drop('event', axis=1)
            championship_rounds = pd.concat([championship_rounds,_championship_rounds]).drop_duplicates()
            
            _events_json = json_normalize(championship_json,['championshipRounds' ])['event']
            _championship_events = json_normalize(_events_json)
            championship_events = pd.concat([championship_events,_championship_events]).drop_duplicates()
        
            _championship_results = _get_single_json_table(meta2, 'championship_results')
            championship_results = pd.concat([championship_results, _championship_results])
    
    for k in championship_entries_all:
        championship_entries_all[k].reset_index(drop=True)
        if k in ['Driver', 'Co-Driver']:
            championship_entries_all[k] = championship_entries_all[k].rename(columns={'TyreManufacturer':'ManufacturerTyre'})
    
    return championship_lookup.reset_index(drop=True), \
            championship_results.reset_index(drop=True), \
            championship_entries_all, \
            championship_rounds.reset_index(drop=True), \
            championship_events.reset_index(drop=True)


In [101]:
#a,b,c,d,e = championship_tables()

## Usage

In [ ]:
#listRallies()

In [29]:
#set_rallyId(meta, 'Monaco')
#meta

{'rallyId': '30', 'rally_name': 'Monaco', 'stages': []}

In [24]:
def cleardbtable(conn, table):
    ''' Clear the table whilst retaining the table definition '''
    c = conn.cursor()
    c.execute('DELETE FROM "{}"'.format(table))
    
def dbfy(conn, df, table, if_exists='append',index=False, clear=False, **kwargs):
    if clear: cleardbtable(conn, table)
    df.to_sql(table,conn,if_exists=if_exists,index=index)

In [25]:
def save_rally(meta, conn):
    
    itinerary_event, itinerary_legs, itinerary_sections, \
    itinerary_stages, itinerary_controls = getItinerary(meta)
    
    dbfy(conn, itinerary_event, 'itinerary_event')
    dbfy(conn, itinerary_legs, 'itinerary_legs')
    dbfy(conn, itinerary_sections, 'itinerary_sections')
    dbfy(conn, itinerary_stages, 'itinerary_stages')
    dbfy(conn, itinerary_controls, 'itinerary_controls')

    startlists, startlist_classes = get_startlists(meta)
    dbfy(conn, startlists, 'startlists')
    dbfy(conn, startlist_classes, 'startlist_classes')

    penalties = get_penalties(meta)
    dbfy(conn, penalties, 'penalties')

    retirements = get_retirements(meta)
    dbfy(conn, retirements, 'retirements')

    stagewinners = get_stagewinners(meta)
    dbfy(conn, stagewinners, 'stagewinners')


    stage_overall = get_overall(meta)
    dbfy(conn, stage_overall, 'stage_overall')

    split_times = get_splitTimes(meta)
    dbfy(conn, split_times, 'split_times')
    
    stage_times_stage = get_stage_times_stage(meta)
    dbfy(conn, stage_times_stage, 'stage_times_stage')
    
    stage_times_overall = get_stage_times_overall(meta)
    dbfy(conn, stage_times_overall, 'stage_times_overall')

    
def save_championship(meta, conn):
    championship_lookup, championship_results, _championship_entries_all, \
        championship_rounds, championship_events = championship_tables()
        
    championship_entries_drivers = _championship_entries_all['Driver']
    championship_entries_codrivers = _championship_entries_all['Co-Driver']
    championship_entries_manufacturers = _championship_entries_all['Manufacturers']
    
    dbfy(conn, championship_lookup, 'championship_lookup', clear=True)
    dbfy(conn, championship_results, 'championship_results', clear=True)
    dbfy(conn, championship_entries_drivers, 'championship_entries_drivers', clear=True)
    dbfy(conn, championship_entries_codrivers, 'championship_entries_codrivers', clear=True)
    dbfy(conn, championship_entries_manufacturers, 'championship_entries_manufacturers', clear=True)
    dbfy(conn, championship_rounds, 'championship_rounds', clear=True)
    dbfy(conn, championship_events, 'championship_events', clear=True)

def get_all(meta, rally, dbname='wrc18_test1.db'):
    
    conn = sqlite3.connect(dbname)
    
    set_rallyId(meta, 'Monaco')
    
    save_rally(meta, conn)
    save_championship(meta, conn)


In [26]:
listRallies()

Monaco, Sweden, Mexico, France, Argentina, Portugal, Italy, Finland, Germany, Turkey, Wales, Spain, Australia


In [27]:
set_rallyId(meta, 'Monaco')

In [32]:
get_all(meta, 'Monaco', dbname='wrc18_test1keysconstraints.db')

In [109]:
dbname='wrc18_test1.db'
conn = sqlite3.connect(dbname)

q="SELECT name FROM sqlite_master WHERE type = 'table';"
pd.read_sql(q,conn)

,name
0,itinerary_event
1,itinerary_legs
2,itinerary_sections
3,itinerary_stages
4,itinerary_controls
5,startlists
6,startlist_classes
7,penalties
8,retirements
9,stagewinners


In [104]:
#!rm wrc18_test1.db